## Task 0
### Exercise 5 Completed by: Natalie Benoy

We will use a dataset that tracks the number of flights between the US and other countries from
2010-2015. The data is split into multiple files, one for each year. The file contains three
columns – destination country, origin country, and flight count. This dataset is pre-loaded in the Databricks File System DBFS. It can be found at: "dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/ "

In [0]:
# import statements
import pyspark.sql.functions as F
from pyspark.sql.types import *

## Task 1

In [0]:
# 1a – View the contents of the data folder using a file system command

In [0]:
%fs
ls dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/

path,name,size,modificationTime
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv,2010-summary.csv,7121,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2011-summary.csv,2011-summary.csv,7069,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2012-summary.csv,2012-summary.csv,6857,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2013-summary.csv,2013-summary.csv,7020,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv,2014-summary.csv,6729,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv,2015-summary.csv,7080,1522192050000


In [0]:
#1b – Use the dbutils module and display function to view directory contents as a table 
display(dbutils.fs.ls("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/"))

path,name,size,modificationTime
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv,2010-summary.csv,7121,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2011-summary.csv,2011-summary.csv,7069,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2012-summary.csv,2012-summary.csv,6857,1522192049000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2013-summary.csv,2013-summary.csv,7020,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv,2014-summary.csv,6729,1522192050000
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv,2015-summary.csv,7080,1522192050000


## Task 2

In [0]:
# 2a – Read the 2015 data in and save the file as flight2015DF
flight2015DF = spark.read.csv("/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv", header = True, sep = ',', inferSchema = False)

In [0]:
# 2b – View the schema in tree format using printSchema() method of Spark DataFrame
flight2015DF.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [0]:
# 2c – set option/parameter inferSchema to True, and re-read 2015 data 
flight2015DF = spark.read.csv("/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv", header = True, sep = ',', inferSchema = True)

In [0]:
# 2d – View the schema in tree format using printSchema() method of Spark DataFrame
flight2015DF.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [0]:
# 2e – Read in another file, 2014 data, and save it in another DF, flight2014DF
flight2014DF = spark.read.csv("/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv", header = True, sep = ',', inferSchema = True)

## Task 3

In [0]:
# 3a – combine the two DFs created in Task 2 for years 2015 and 2014 together using the union() method of Spark DataFrame.
flight20142015DF = flight2014DF.union(flight2015DF)

In [0]:
# 3b – Print the number of rows from combining 2015 and 2104 files
flight20142015DF.count()

Out[10]: 497

In [0]:
# 3c – Combine data from all six input .csv files for 2010-2015
flightsAllDF = spark.read.csv("/databricks-datasets/definitive-guide/data/flight-data/csv/*.csv", header = True, sep = ',', inferSchema = True)

In [0]:
flightsAllDF = spark.read.format('csv').load("/databricks-datasets/definitive-guide/data/flight-data/csv/", header = True, sep = ',', inferSchema = True)

In [0]:
flightsAllDF.count()

Out[15]: 1502

## Task 4

In [0]:
# 4a – Read/load all *.csv files by specifying a custom schema instead of inferring it.
myschema = StructType([
    StructField("dest", StringType()),
    StructField("origin", StringType()),
    StructField("count", LongType())])

flightsAllDF = spark.read.csv("/databricks-datasets/definitive-guide/data/flight-data/csv/*.csv", header = True, sep = ',', schema = myschema)

In [0]:
# 4b – Print the schema in tree format using printSchema() method of Spark DataFrame.
flightsAllDF.printSchema()

root
 |-- dest: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- count: long (nullable = true)



In [0]:
# 4c – Output the count of rows in flightsAllDF and the count of distinct rows in flightsAllDF and explain the difference
print(flightsAllDF.count())
print(flightsAllDF.distinct().count())

1502
1328


These datasets were originally organized by year. When we combined them into one dataset, we lost the year information, which means that flights between the same two countries over different years are currently treated as duplicates.

## Task 5

In [0]:
# 5a – Use an action command to view all the records of the DataFrame flightsAllDF at the Driver node.
flightsAllDF.collect()

Out[38]: [Row(dest='United States', origin='Romania', numflights=1, year=2010),
 Row(dest='United States', origin='Ireland', numflights=264, year=2010),
 Row(dest='United States', origin='India', numflights=69, year=2010),
 Row(dest='Egypt', origin='United States', numflights=24, year=2010),
 Row(dest='Equatorial Guinea', origin='United States', numflights=1, year=2010),
 Row(dest='United States', origin='Singapore', numflights=25, year=2010),
 Row(dest='United States', origin='Grenada', numflights=54, year=2010),
 Row(dest='Costa Rica', origin='United States', numflights=477, year=2010),
 Row(dest='Senegal', origin='United States', numflights=29, year=2010),
 Row(dest='United States', origin='Marshall Islands', numflights=44, year=2010),
 Row(dest='Guyana', origin='United States', numflights=17, year=2010),
 Row(dest='United States', origin='Sint Maarten', numflights=53, year=2010),
 Row(dest='Malta', origin='United States', numflights=1, year=2010),
 Row(dest='Bolivia', origin='Unite

In [0]:
# 5b – Use three different action commands to view just three rows of the distributed DF
flightsAllDF.show(n=3)
flightsAllDF.limit(3).show()
flightsAllDF.head(3)

+-------------+-------+-----+
|         dest| origin|count|
+-------------+-------+-----+
|United States|Romania|    1|
|United States|Ireland|  264|
|United States|  India|   69|
+-------------+-------+-----+
only showing top 3 rows

+-------------+-------+-----+
|         dest| origin|count|
+-------------+-------+-----+
|United States|Romania|    1|
|United States|Ireland|  264|
|United States|  India|   69|
+-------------+-------+-----+

Out[16]: [Row(dest='United States', origin='Romania', count=1),
 Row(dest='United States', origin='Ireland', count=264),
 Row(dest='United States', origin='India', count=69)]

In [0]:
# 5c – Print the type of the first row of the DataFrame. 
type(flightsAllDF.first())

Out[17]: pyspark.sql.types.Row

## Task 6

In [0]:
# 6a – Create a new DF (say flightsAllDF_1) with two changes as described below – rename an existing column ‘count’ to ‘numflights’, and add a new column with the filename as values
flightsAllDF_1 = flightsAllDF.withColumnRenamed("count", "numflights").withColumn("filename", F.input_file_name())

In [0]:
# 6b – Repeat task 6a. using select() method of Spark DataFrame and column expressions
flightsAllDF_1 = flightsAllDF.select('dest', "origin", F.col("count").alias("numflights"), F.input_file_name().alias("filename"))

In [0]:
# 6c – Output 5 records showing just the numflights and filename columns in table form
flightsAllDF_1.select("numflights", "filename").show(5, truncate = False)

+----------+--------------------------------------------------------------------------------+
|numflights|filename                                                                        |
+----------+--------------------------------------------------------------------------------+
|1         |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|264       |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|69        |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|24        |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
|1         |dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv|
+----------+--------------------------------------------------------------------------------+
only showing top 5 rows



In [0]:
# 6d – Print out just the column names of FlightsAllDF (there should only be four column names).
flightsAllDF = flightsAllDF_1

flightsAllDF.columns

Out[8]: ['dest', 'origin', 'numflights', 'filename']

## Task 7

In [0]:
# 7a – Add a new column to flightsAllDF called ‘year’ (cast as type integer), and populate it with just the year extracted from the column filename above. Save this is a new DF called flightsAllDF_2 
flightsAllDF_2 = flightsAllDF.select('*', F.regexp_extract('filename', '/(\d+)-', 1).alias("year").cast(IntegerType()))

flightsAllDF_2.show(5)

+-----------------+-------------+----------+--------------------+----+
|             dest|       origin|numflights|            filename|year|
+-----------------+-------------+----------+--------------------+----+
|    United States|      Romania|         1|dbfs:/databricks-...|2010|
|    United States|      Ireland|       264|dbfs:/databricks-...|2010|
|    United States|        India|        69|dbfs:/databricks-...|2010|
|            Egypt|United States|        24|dbfs:/databricks-...|2010|
|Equatorial Guinea|United States|         1|dbfs:/databricks-...|2010|
+-----------------+-------------+----------+--------------------+----+
only showing top 5 rows



In [0]:
# 7b – Re-save this new DF flightsAllDF_2 back in flightsAllDF. Drop column 'filename'
flightsAllDF = flightsAllDF_2.drop("filename")

In [0]:
# 7c – View 5 records in table format showing all columns of flightsAllDF
flightsAllDF.show(5)

+-----------------+-------------+----------+----+
|             dest|       origin|numflights|year|
+-----------------+-------------+----------+----+
|    United States|      Romania|         1|2010|
|    United States|      Ireland|       264|2010|
|    United States|        India|        69|2010|
|            Egypt|United States|        24|2010|
|Equatorial Guinea|United States|         1|2010|
+-----------------+-------------+----------+----+
only showing top 5 rows



In [0]:
# 7d – View the data types of the columns of flightsAllDF using a DataFrame attribute
flightsAllDF.dtypes

Out[25]: [('dest', 'string'),
 ('origin', 'string'),
 ('numflights', 'bigint'),
 ('year', 'int')]

## Task 8

In [0]:
# 8a – sort flightsAllDF by ‘dest’ in ascending order and ‘numflights’ in descending order. View the top 10 records
flightsAllDF.orderBy(["dest", "numflights"], ascending = [True, False]).show(10)

+-----------+-------------+----------+----+
|       dest|       origin|numflights|year|
+-----------+-------------+----------+----+
|Afghanistan|United States|        11|2010|
|Afghanistan|United States|         8|2011|
|Afghanistan|United States|         5|2012|
|    Algeria|United States|         9|2014|
|    Algeria|United States|         4|2015|
|    Algeria|United States|         2|2013|
|     Angola|United States|        15|2015|
|     Angola|United States|        14|2010|
|     Angola|United States|        13|2014|
|     Angola|United States|        13|2011|
+-----------+-------------+----------+----+
only showing top 10 rows



## Task 9

In [0]:
# 9a – How many distinct destinations are in the dataset?
flightsAllDF.select(F.countDistinct("dest")).show()

+--------------------+
|count(DISTINCT dest)|
+--------------------+
|                 167|
+--------------------+



In [0]:
# 9b – How many distinct combinations of destination and year are there?
print(flightsAllDF.select("dest", "year").distinct().count())

772


## Task 10

In [0]:
# 10a – How many records of flightsAllDF contain either Japan or South Korea as the destination?
print(flightsAllDF.filter((F.col("dest") == 'Japan') | (F.col("dest") == 'South Korea')).count())

12


In [0]:
# 10b – How many total flights did United States send to South Korea in 2013? 
flightsAllDF.filter((F.col("origin") == 'United States') & (F.col("dest") == 'South Korea') & (F.col("year") == 2013)).show()

# total flights is 842

+-----------+-------------+----------+----+
|       dest|       origin|numflights|year|
+-----------+-------------+----------+----+
|South Korea|United States|       842|2013|
+-----------+-------------+----------+----+



## Task 11

In [0]:
# 11a – Display a bar chart of the number of domestic flights(Y) vs year(X), sorted by year in ascending order.
display(flightsAllDF.filter((F.col("origin") == 'United States') & (F.col("dest") == 'United States')).sort("year"))

dest,origin,numflights,year
United States,United States,348113,2010
United States,United States,352742,2011
United States,United States,347452,2012
United States,United States,343132,2013
United States,United States,358354,2014
United States,United States,370002,2015


## Task 12

In [0]:
# 12a – How many total flights (domestic and international) are flown in each of the 6 years? Print the output sorted in descending order of the sum of number of flights column.
flightsAllDF.groupby("year").sum("numflights").sort("sum(numflights)", ascending = False).show()

+----+---------------+
|year|sum(numflights)|
+----+---------------+
|2015|         453316|
|2014|         436855|
|2011|         428426|
|2010|         422269|
|2012|         422180|
|2013|         417869|
+----+---------------+



## Task 13

In [0]:
# 13a – What is the total number of international flights that flew to the United States in each of the 6 years?
# Obtain a table with the column of flight count values named as totalflightsin instead of the default, and sort the results in ascending order of totalflightsin.
inUSADF = flightsAllDF.filter((F.col("origin") != 'United States') & (F.col("dest") == 'United States')).groupby("year").sum("numflights").withColumnRenamed("sum(numflights)", "totalflightsin")

display(inUSADF.sort("totalflightsin"))

year,totalflightsin
2010,36819
2013,36830
2012,36890
2011,37763
2014,38833
2015,41350


In [0]:
# 13b – In what years were there more international flights flying to the United States than international flights that flew out of the United States? Include:
# i. year – year of data, in ascending order
# ii. totalflightsin – total number of international flights to the US as destination
# iii. totalflightout - total number of international flights from the US as origin
# iv. morein – a boolean that is set to True if totalflightsin > totalflightsout

outUSADF = flightsAllDF.filter((F.col("origin") == 'United States') & (F.col("dest") != 'United States')).groupby("year").sum("numflights").withColumnRenamed("sum(numflights)", "totalflightsout")

USADF = inUSADF.join(outUSADF, on = 'year', how = 'inner')

USADF.select('*', F.col("totalflightsin") > F.col("totalflightsout")).withColumnRenamed("(totalflightsin > totalflightsout)", "morein").sort("year").show()

+----+--------------+---------------+------+
|year|totalflightsin|totalflightsout|morein|
+----+--------------+---------------+------+
|2010|         36819|          37337| false|
|2011|         37763|          37921| false|
|2012|         36890|          37838| false|
|2013|         36830|          37907| false|
|2014|         38833|          39668| false|
|2015|         41350|          41964| false|
+----+--------------+---------------+------+



## Task 14

In [0]:
# 14a – For the rows that contain United States as destination but NOT as origin (i.e. international flights into the USA) compute and print the following summary statistics for the column numflights – sum, avg, max - for each of the 6 years.
# The output should have columns named, year, sumflightstoUS, avgflightstoUS, maxflightstoUS.
# Ascending sort the DataFrame by year. 

flightsAllDF.filter((F.col("origin") != 'United States') & (F.col("dest") == 'United States')).groupby("year").agg(F.sum("numflights").alias("sumflightstoUS"), F.mean("numflights").alias("avgflightstoUS"), F.max("numflights").alias("maxflightstoUS")).sort("year").show()

+----+--------------+------------------+--------------+
|year|sumflightstoUS|    avgflightstoUS|maxflightstoUS|
+----+--------------+------------------+--------------+
|2010|         36819| 283.2230769230769|          8305|
|2011|         37763| 299.7063492063492|          8650|
|2012|         36890| 312.6271186440678|          8097|
|2013|         36830| 320.2608695652174|          7983|
|2014|         38833| 331.9059829059829|          8177|
|2015|         41350|333.46774193548384|          8483|
+----+--------------+------------------+--------------+

